<a href="https://colab.research.google.com/github/rahiakela/automl-experiments/blob/main/auto-keras-practice-works/04_text_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Text Regression

Reference:

https://autokeras.com/tutorial/text_regression/

In [1]:
!pip -q install autokeras

     |████████████████████████████████| 166 kB 5.1 MB/s 
     |████████████████████████████████| 97 kB 4.8 MB/s 
     |████████████████████████████████| 454.4 MB 19 kB/s 
     |████████████████████████████████| 462 kB 39.8 MB/s 
     |████████████████████████████████| 4.0 MB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 22.2 MB/s 


In [2]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

To make this tutorial easy to follow, we just treat IMDB dataset as a regression dataset. It means we will treat prediction targets of IMDB dataset, which are 0s and 1s as numerical values, so that they can be directly used as the regression targets.

##A Simple Example

The first step is to prepare your data. Here we use the IMDB dataset as an example.

In [3]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

# set path to dataset
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # <START> this film was just brilliant casting <UNK>

84131840/84125825 [==============================] - 7s 0us/step
(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


The second step is to run the TextRegressor. As a quick demo, we set epochs to 2. You can also leave the epochs unspecified for an adaptive number of epochs.

In [4]:
# Initialize the text regressor.
reg = ak.TextRegressor(overwrite=True, max_trials=1)

# Feed the text regressor with training data.
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 01m 29s]
val_loss: 0.15182562172412872

Best val_loss So Far: 0.15182562172412872
Total elapsed time: 00h 01m 29s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 30s 37ms/step - loss: 0.1847 - mean_squared_error: 0.1847
Epoch 2/2
782/782 [==============================] - 30s 39ms/step - loss: 0.1235 - mean_squared_error: 0.1235
INFO:tensorflow:Assets written to: ./text_regressor/best_model/assets


In [5]:
# Predict with the best model
predicted_y = reg.predict(x_test)
# Evaluate the best model with testing data
print(reg.evaluate(x_test, y_test))

782/782 [==============================] - 7s 9ms/step - loss: 0.1533 - mean_squared_error: 0.1533
[0.15334804356098175, 0.15334804356098175]


##Validation Data

By default, AutoKeras use the last 20% of training data as validation data. As shown in the example below, you can use `validation_split` to specify the percentage.

In [ ]:
reg.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
)

You can also use your own validation set instead of splitting it from the training data with `validation_data`.

In [ ]:
split = 5000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]

reg.fit(
    x_train,
    y_train,
    epochs=2,
    # Use your own validation set.
    validation_data=(x_val, y_val),
)

##Customized Search Space

For advanced users, you may customize your search space by using AutoModel instead of TextRegressor. You can configure the TextBlock for some high-level configurations, e.g., vectorizer for the type of text vectorization method to use. You can use 'sequence', which uses TextToInteSequence to convert the words to integers and use Embedding for embedding the integer sequences, or you can use 'ngram', which uses TextToNgramVector to vectorize the sentences. You can also do not specify these arguments, which would leave the different choices to be tuned automatically.

In [6]:
input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 51s]
val_loss: 0.4516303837299347

Best val_loss So Far: 0.4516303837299347
Total elapsed time: 00h 00m 51s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 10s 11ms/step - loss: 0.6867 - mean_squared_error: 0.6867
Epoch 2/2
782/782 [==============================] - 9s 12ms/step - loss: 0.1292 - mean_squared_error: 0.1292
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


The usage of AutoModel is similar to the functional API of Keras. Basically, you are building a graph, whose edges are blocks and the nodes are intermediate outputs of blocks. To add an edge from input_node to `output_node` with `output_node = ak.[some_block]([block_args])(input_node)`.

You can even also use more fine grained blocks to customize the search space even further.

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)
reg.fit(x_train, y_train, epochs=2)

##Data Format

The AutoKeras TextRegressor is quite flexible for the data format.

For the text, the input data should be one-dimensional For the regression targets, it should be a vector of numerical values. AutoKeras accepts numpy.ndarray.

We also support using tf.data.Dataset format for the training data.

In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,))).batch(32)

reg = ak.TextRegressor(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=2)
# Predict with the best model.
predicted_y = reg.predict(test_set)
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))